In [1]:
# this cell enables relative path imports
import os
from dotenv import load_dotenv
import pickle
load_dotenv()
_PROJECT_PATH: str = os.environ["_project_path"]
_PICKLED_DATA_FILENAME: str = os.environ["_pickled_data_filename"]
_CSV_DATA_FILENAME: str = os.environ["_csv_data_filename"]



import sys
from pathlib import Path
project_path = Path(_PROJECT_PATH)
sys.path.append(str(project_path))

In [2]:
# import all your modules here
import time
import json
import pandas as pd

import config_v2 as cfg
from library_ubidots_v2 import Ubidots as ubi

Baseline en config_v2.py: ['2023-01-01', '2025-01-20']
Study en config_v2.py: ['2025-01-20', '2025-01-27']


In [3]:
#blob_name = r"data/data_weekly_report.pkl"
#downloader.upload_pkl_file(bancolombia_pkl,blob_name, blob_container)

In [4]:
# set your constants here
baseline=cfg.BASELINE
study=cfg.STUDY

In [5]:
print(study)
print(baseline)

['2025-01-20', '2025-01-27']
['2023-01-01', '2025-01-20']


In [6]:
df_devices = ubi.get_available_devices_v2(label='bancolombia-f2', level='group', page_size=1000)
df_vars = ubi.get_available_variables(list(df_devices['device_id']))

In [7]:
df_vars = df_vars[df_vars['variable_label'].isin(cfg.WHITELISTED_VAR_LABELS)]
VAR_IDS_TO_REQUEST = list(df_vars['variable_id'])
VAR_ID_TO_LABEL = dict(zip(df_vars['variable_id'], df_vars['variable_label']))

In [8]:
#leer el archivo data_weekly_report.pkl y guardarlo en un dataframe llamado df1
df1 = pd.read_pickle(r'C:\Digitalización\Bancolombia\CB_informe_Fase_2\data\data_weekly_report.pkl')


In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 22615720 entries, 2024-10-07 00:00:00-05:00 to 2025-01-13 00:00:00-05:00
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   value        float64
 1   variable     object 
 2   device       object 
 3   device_name  object 
dtypes: float64(1), object(3)
memory usage: 862.7+ MB


In [10]:
CHUNK_SIZE = 10
DATE_INTERVAL_REQUEST = {'start': study[0], 'end': study[1]}
df = None
lst_responses = []
n_vars = len(VAR_IDS_TO_REQUEST)
print(f"Making request for the following interval: Study:{study[0]}, Study:{study[1]}")
for idx in range(0, ubi.ceildiv(len(VAR_IDS_TO_REQUEST), CHUNK_SIZE)):
    idx_start = idx * CHUNK_SIZE
    idx_end = (idx + 1) * CHUNK_SIZE
    chunk = VAR_IDS_TO_REQUEST[idx_start:idx_end]
    response = ubi.make_request(
        chunk, 
        DATE_INTERVAL_REQUEST, 
    )
    if response.status_code == 204 or response.status_code >= 500:
        print(f"Empty response for chunk {idx}")
        time.sleep(10)
        response = ubi.make_request(
        chunk, 
        DATE_INTERVAL_REQUEST,)
    current_idx = idx_end+1
    if (current_idx > n_vars):
        current_idx = n_vars
    print(f"Progress: {100*(current_idx)/n_vars:0.1f}%")
    print(f"Response status code: {response.status_code}")
    if (response.status_code != 204) and  (len(response.json()['results']) >0 ):
        lst_responses.append(response)
    else: 
        print(f"Empty response for chunk {idx}")
df = ubi.parse_response(lst_responses, VAR_ID_TO_LABEL)

Making request for the following interval: Study:2025-01-20, Study:2025-01-27
Progress: 3.0%
Response status code: 200
Progress: 5.7%
Response status code: 200
Progress: 8.5%
Response status code: 200
Progress: 11.2%
Response status code: 200
Progress: 13.9%
Response status code: 200
Progress: 16.7%
Response status code: 200
Progress: 19.4%
Response status code: 200
Progress: 22.1%
Response status code: 200
Progress: 24.9%
Response status code: 200
Progress: 27.6%
Response status code: 200
Progress: 30.3%
Response status code: 200
Progress: 33.1%
Response status code: 200
Progress: 35.8%
Response status code: 200
Progress: 38.5%
Response status code: 200
Progress: 41.3%
Response status code: 200
Progress: 44.0%
Response status code: 200
Progress: 46.7%
Response status code: 200
Progress: 49.5%
Response status code: 200
Progress: 52.2%
Response status code: 200
Progress: 54.9%
Response status code: 200
Progress: 57.7%
Response status code: 200
Progress: 60.4%
Response status code: 200
P

In [11]:
df.sort_index()

,value,variable,device,device_name
datetime,,,,
2025-01-20 00:00:00-05:00,0.400000,front-potencia-activa,bc80t,BC 689 - Metropolis
2025-01-20 00:00:00-05:00,0.990000,factor-de-potencia,bc89t,BC 384 - Anapoima
2025-01-20 00:00:00-05:00,8.000000,front-consumo-activa,bc---metropolitan,BC - Metropolitan
2025-01-20 00:00:00-05:00,0.038333,kw-tr,bc-832---cucuta,BC 832 - San Francisco de Paula
2025-01-20 00:00:00-05:00,4.250000,aa-consumo-activa,bc---metropolitan,BC - Metropolitan
...,...,...,...,...
2025-01-27 00:00:00-05:00,2.840000,front-consumo-activa,bc-478---mix-via-40,BC 478 - Mix Vía 40
2025-01-27 00:00:00-05:00,0.000000,ilu-consumo-activa,bc-673---calle-80,BC 673 - Calle 80
2025-01-27 00:00:00-05:00,0.440000,aa-consumo-activa,bc-673---calle-80,BC 673 - Calle 80


In [12]:
df1.sort_index()

,value,variable,device,device_name
datetime,,,,
2023-12-01 00:00:00-05:00,2.600000,front-potencia-activa,bc85t,BC 802 - Puerto Lopez
2023-12-01 00:00:00-05:00,2.000000,front-consumo-activa,bc80t,BC 689 - Metropolis
2023-12-01 00:00:00-05:00,7.000000,front-consumo-activa,bc90t,BC 388 - CC Hayuelos
2023-12-01 00:00:00-05:00,2.100000,aa-consumo-activa,bc88t,BC 332 - Zipaquira
2023-12-01 00:00:00-05:00,0.730000,aa-consumo-activa,bc80t,BC 689 - Metropolis
...,...,...,...,...
2025-01-20 00:00:00-05:00,0.020779,kw-tr,bc85t,BC 802 - Puerto Lopez
2025-01-20 00:00:00-05:00,235.840000,area,bc-513---el-dificil,BC 513 - El Dificil
2025-01-20 00:00:00-05:00,242.300000,area,bc-111---corozal,BC 111 - Corozal


In [13]:
#Unir el dataframe dfprogress: 30.5%Response status code: 200Progress: 34.8%Response status code: 200Progress: 39.1%Response status code: 200Progress: 43.3%Response status code: 200 y df1 en uno que se llame df_full / esto es para mas adelante
df = pd.concat([df1, df], axis=0, ignore_index=False)

# para quitar el indixe de "datatime" y volverlo columna 
df.reset_index(inplace=True)

# Quitar duplicados 
df = df.drop_duplicates()

# Establecer la columna "datetime" como el índice
df.set_index('datetime', inplace=True)


In [14]:
pd.to_pickle(df, project_path / 'data'/ _PICKLED_DATA_FILENAME)

In [15]:
df

,value,variable,device,device_name
datetime,,,,
2024-10-07 00:00:00-05:00,11.2,front-potencia-activa,bc---metropolitan,BC - Metropolitan
2024-10-06 23:57:00-05:00,10.8,front-potencia-activa,bc---metropolitan,BC - Metropolitan
2024-10-06 23:54:00-05:00,7.0,front-potencia-activa,bc---metropolitan,BC - Metropolitan
2024-10-06 23:51:00-05:00,7.0,front-potencia-activa,bc---metropolitan,BC - Metropolitan
2024-10-06 23:48:00-05:00,6.8,front-potencia-activa,bc---metropolitan,BC - Metropolitan
...,...,...,...,...
2025-01-20 00:15:00-05:00,0.4,front-potencia-activa,bc80t,BC 689 - Metropolis
2025-01-20 00:12:00-05:00,0.4,front-potencia-activa,bc80t,BC 689 - Metropolis
2025-01-20 00:09:00-05:00,0.4,front-potencia-activa,bc80t,BC 689 - Metropolis
